In [1]:
import zipfile
import os

# Unzip the dataset
with zipfile.ZipFile('../../data/cryptopunks.zip', 'r') as zip_ref:
    zip_ref.extractall('../../data')

In [2]:
import numpy as np 

In [13]:
from PIL import Image
from numpy import asarray
import torch 

image = Image.open('../../data/imgs/0.png')


In [15]:
image.size

(24, 24)

In [16]:
image = torch.tensor(np.array(image))

image = image.view(image.size(0), -1)

In [22]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Define the transformations
transform = transforms.Compose([
    transforms.ToTensor(),          # Convert images to PyTorch tensors
])

import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import glob

class UnlabeledDataset(Dataset):
    def __init__(self, root_dir='../../data/imgs', transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = glob.glob(os.path.join(root_dir, '*.png'))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image

# Define the transformations
transform = transforms.Compose([
    transforms.ToTensor(),          
])

# Load the dataset
dataset = UnlabeledDataset(transform=transform)

In [23]:
# Split the dataset into training and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [24]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [26]:
from autoencoder import Autoencoder
import torch.nn as nn 

In [32]:
autoencoder = Autoencoder(input_dim = 24*24, hidden_dim=128, latent_dim=64)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-3)

# Training loop
epochs = 20
for epoch in range(epochs):
    for data in train_loader:
        img = data
        # img = img.view(img.size(0), -1)  

        # Forward pass
        output = autoencoder(img)
        loss = criterion(output, img)
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (4608x24 and 576x128)

In [30]:
img.shape

torch.Size([64, 3, 24, 24])